In [1]:
%run init.ipynb

C:\Users\Administrator\Anaconda3\lib\requests\__init__.py:80: RequestsDependencyWarning: urllib3 (1.25.11) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


matchzoo version 1.0.2
`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]
data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`
停用表配置成功


In [2]:
preprocessor = mz.models.DRMM.get_default_preprocessor()

In [3]:
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
dev_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

Processing text_left with chain_transform of ChineseRemoveBlack => ChineseSimplified => ChineseEmotion => IsChinese => ChineseStopRemoval => ChineseTokenizeDemo => Tokenize => Lowercase => PuncRemoval:   0%| | 0/90 [00:00<?, ?it/s]Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 1.480 seconds.
Prefix dict has been built succesfully.
Processing text_left with chain_transform of ChineseRemoveBlack => ChineseSimplified => ChineseEmotion => IsChinese => ChineseStopRemoval => ChineseTokenizeDemo => Tokenize => Lowercase => PuncRemoval: 100%|█| 90/90 [00:02<00:00, 38.29it/s]
Processing text_right with chain_transform of ChineseRemoveBlack => ChineseSimplified => ChineseEmotion => IsChinese => ChineseStopRemoval => ChineseTokenizeDemo => Tokenize => Lowercase => PuncRemoval: 100%|█| 92/92 [00:00<00:00, 154.97it/s]
Building Vocabulary from a datapack.: 100%|██████████████████████████████████████| 7

In [4]:
preprocessor.context

{'embedding_input_dim': 286,
 'filter_unit': <mzcn.preprocessors.units.frequency_filter.FrequencyFilter at 0x14d10cbfb38>,
 'vocab_size': 286,
 'vocab_unit': <mzcn.preprocessors.units.vocabulary.Vocabulary at 0x14d1e83c860>}

In [5]:
# 因为离不开embedding_matrix的生成，所以我直接在.matchzoo的文件夹里面直接解压了对应的数据集

In [6]:
glove_embedding = mz.datasets.embeddings.load_glove_embedding(dimension=50)
term_index = preprocessor.context['vocab_unit'].state['term_index']
embedding_matrix = glove_embedding.build_matrix(term_index)
l2_norm = np.sqrt((embedding_matrix * embedding_matrix).sum(axis=1))
embedding_matrix = embedding_matrix / l2_norm[:, np.newaxis]

In [7]:
import gc
gc.collect()

0

In [8]:
histgram_callback = mz.dataloader.callbacks.Histogram(
    embedding_matrix, bin_size=30, hist_mode='LCH'
)

trainset = mz.dataloader.Dataset(
    data_pack=train_pack_processed,
    mode='pair',
    num_dup=2,
    num_neg=1,
    callbacks=[histgram_callback]
)
devset = mz.dataloader.Dataset(
    data_pack=dev_pack_processed,
    callbacks=[histgram_callback]
)

In [9]:
padding_callback = mz.models.DRMM.get_default_padding_callback()

trainloader = mz.dataloader.DataLoader(
    device='cpu',
    dataset=trainset,
    stage='train',
    callback=padding_callback,
)
devloader = mz.dataloader.DataLoader(
    dataset=devset,
    stage='dev',
    callback=padding_callback,
)

In [10]:
model = mz.models.DRMM()

model.params['task'] = ranking_task
model.params['mask_value'] = 0
# model.params['embedding'] = embedding_matrix #这里是当加载glove等模型时取消该行注释
#设置embedding系数
model.params["embedding_output_dim"]=100
model.params["embedding_input_dim"]=preprocessor.context["embedding_input_dim"]
model.params['hist_bin_size'] = 30
model.params['mlp_num_layers'] = 1
model.params['mlp_num_units'] = 10
model.params['mlp_num_fan_out'] = 1
model.params['mlp_activation_func'] = 'tanh'

model.build()

print(model)
print('Trainable params: ', sum(p.numel() for p in model.parameters() if p.requires_grad))

DRMM(
  (embedding): Embedding(286, 100, padding_idx=0)
  (attention): Attention(
    (linear): Linear(in_features=100, out_features=1, bias=False)
  )
  (mlp): Sequential(
    (0): Sequential(
      (0): Linear(in_features=30, out_features=10, bias=True)
      (1): Tanh()
    )
    (1): Sequential(
      (0): Linear(in_features=10, out_features=1, bias=True)
      (1): Tanh()
    )
  )
  (out): Linear(in_features=1, out_features=1, bias=True)
)
Trainable params:  29023


In [11]:
optimizer = torch.optim.Adadelta(model.parameters())

trainer = mz.trainers.Trainer(
    device='cpu',
    model=model,
    optimizer=optimizer,
    trainloader=trainloader,
    validloader=devloader,
    validate_interval=None,
    epochs=5
)
trainer.run()

[Iter-1 Loss-0.999]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.1923 - normalized_discounted_cumulative_gain@5(0.0): 0.1923 - mean_average_precision(0.0): 0.1923



[Iter-2 Loss-0.999]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.1923 - normalized_discounted_cumulative_gain@5(0.0): 0.1923 - mean_average_precision(0.0): 0.1923



[Iter-3 Loss-0.999]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.1923 - normalized_discounted_cumulative_gain@5(0.0): 0.1923 - mean_average_precision(0.0): 0.1923



[Iter-4 Loss-0.998]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.1923 - normalized_discounted_cumulative_gain@5(0.0): 0.1923 - mean_average_precision(0.0): 0.1923



[Iter-5 Loss-0.998]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.1923 - normalized_discounted_cumulative_gain@5(0.0): 0.1923 - mean_average_precision(0.0): 0.1923

Cost time: 2.0288355350494385s


In [12]:
import gc
gc.collect()

617